In [1]:
import numpy as np
import pprint
import json

Step 1: Calculate expected side products (without affecting benzoate part)

In [2]:
Ol2Et = {
    'st_ms' : 184.15,
    'updt_ms' : 168.324,
    'rxn' : 'smOH->OC12H25_',
    'max_rxn' : 3,
    'msg' : '* OH group(s) on trioxyl methyl-benzoate substituted by dodecyl group(s). '
}

In [3]:
all_ms_Ol2Et = {
    'rxn_n' : [],
    'ms' : [],
    'msg' : [],
    'name' : []
}

def all_ms_updt(all_ms_dict, new_rxn_n, new_ms, new_msg, rxn_type, pre_rxn = ''):
    all_ms_dict['rxn_n'].append(new_rxn_n)
    all_ms_dict['ms'].append(new_ms)
    all_ms_dict['msg'].append(new_msg)
    new_name = pre_rxn + rxn_type + str(new_rxn_n)
    all_ms_dict['name'].append(new_name)
    # print(all_ms_dict)
    return all_ms_dict

# calculate all the possible combinations with OH substitution

for i in range(Ol2Et['max_rxn'] + 1):
    new_ms = Ol2Et['st_ms'] + Ol2Et['updt_ms'] * i
    # print(new_ms)
    new_msg = str(i) + ' ' + Ol2Et['msg']
    # print(new_msg)
    rxn_type = Ol2Et['rxn']
    all_ms_updt(all_ms_Ol2Et, i, new_ms, new_msg, rxn_type, pre_rxn = '')

pprint.pprint(all_ms_Ol2Et)
    


{'ms': [184.15, 352.47400000000005, 520.798, 689.1220000000001],
 'msg': ['0 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '1 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '2 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). ',
         '3 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). '],
 'name': ['smOH->OC12H25_0',
          'smOH->OC12H25_1',
          'smOH->OC12H25_2',
          'smOH->OC12H25_3'],
 'rxn_n': [0, 1, 2, 3]}


Step 2: Work out potential products generated by ester hydrolysis from the previous structures.

In [4]:
Est2Acd = {
    'st_ms_lst' : all_ms_Ol2Et['ms'],
    'updt_ms' : -14.027,
    'rxn' : '|COOMe->COOH_',
    'max_rxn' : 1,
    'msg' : '* ester group(s) on methyl-benzoate hydrolysised into carboxylic acid. '
}

all_ms_Est2Acd = {
    'rxn_n' : [],
    'ms' : [],
    'msg' : [],
    'name' : []
}

# calculate all the possible combinations with follow-up ester hydrolysis

for i in range(Est2Acd['max_rxn'] + 1):

    for k, et_ms in enumerate(Est2Acd['st_ms_lst']):

        new_ms = et_ms + Est2Acd['updt_ms'] * i
        # print(new_ms)
        pre_msg = all_ms_Ol2Et['msg'][k]
        new_msg = pre_msg + str(i) + ' ' + Est2Acd['msg']
        # print(new_msg)
        rxn_type = Est2Acd['rxn']
        pre_rxn = all_ms_Ol2Et['name'][k]
        all_ms_updt(all_ms_Est2Acd, i, new_ms, new_msg, rxn_type, pre_rxn = pre_rxn)

all_ms_Est2Acd['ms_with_Na'] = [x + 23.990 for x in all_ms_Est2Acd['ms']]
pprint.pprint(all_ms_Est2Acd)

{'ms': [184.15,
        352.47400000000005,
        520.798,
        689.1220000000001,
        170.12300000000002,
        338.44700000000006,
        506.771,
        675.095],
 'msg': ['0 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). 0 * ester group(s) on methyl-benzoate hydrolysised into '
         'carboxylic acid. ',
         '1 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). 0 * ester group(s) on methyl-benzoate hydrolysised into '
         'carboxylic acid. ',
         '2 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). 0 * ester group(s) on methyl-benzoate hydrolysised into '
         'carboxylic acid. ',
         '3 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '
         'group(s). 0 * ester group(s) on methyl-benzoate hydrolysised into '
         'carboxylic acid. ',
         '0 * OH group(s) on trioxyl methyl-benzoate substituted by dodecyl '

Step 3: Predict side reactions that may occur between suspicious solvents.

In [5]:
# potential side rxns triggered by solvents

solv_N_lp = ['NMI', 'Pyd', 'TEA'] # solvents with lone pairs on N can possibly form ionic liquid with BrAlkane or attach at the ester/acid site. ACN requires further check.
solv_OH = ['nPA', 'EtOH', 'IPA', 'tBuOH'] # solvents with OH groups can possibly form ether with BrAlkane via SN2 pathway and E2 product with the alcohol itself. It may also trigger transesterification at the ester/acid site.
solv_est = ['EA'] # solvents contain ester group may trigger transesterification at the ester/acid site.

solv_side_rxn = {
    'N_lp' : ['IL', 'EstAtk'],
    'OH' : ['SN2E2', 'EstAtk'],
    'est' : ['EstExg']
}

In [6]:
ms_BrAlk = 249.24

# open solvents with suspcious properties

f = open('susp_solv.json')
susp_solv_data = json.load(f)

pprint.pprint(susp_solv_data)

[{'full_name': 'N-methylimidazole',
  'ms': 82.1,
  'name': 'NMI',
  'susp_prop': 'N_lp'},
 {'full_name': 'Pyridine', 'ms': 79.1, 'name': 'Pyd', 'susp_prop': 'N_lp'},
 {'full_name': 'Triethylamine',
  'ms': 101.19,
  'name': 'TEA',
  'susp_prop': 'N_lp'},
 {'full_name': '1-propanol', 'ms': 60.09, 'name': 'nPA', 'susp_prop': 'OH'},
 {'full_name': 'ethanol', 'ms': 46.07, 'name': 'EtOH', 'susp_prop': 'OH'},
 {'full_name': 'isopropanol', 'ms': 60.1, 'name': 'IPA', 'susp_prop': 'OH'},
 {'full_name': 'tertbutylalcohol',
  'ms': 74.12,
  'name': 'tBuOH',
  'susp_prop': 'OH'},
 {'exg_frag_ms': 46.01,
  'full_name': 'ethyal_acetate',
  'ms': 88.11,
  'name': 'EA',
  'susp_prop': 'est'}]


In [7]:
side_prod_IL = {

    'side_rx_type' : 'IL',
    'ms' : [],
    'msg' : [],
    'name' : []

}


def side_rx_IL(susp_solv):

    prod_name = susp_solv['name'] + '_' + 'BrAlkane' + '_salt'
    prod_ms = susp_solv['ms'] + ms_BrAlk
    prod_msg = 'Ionic liuqid is formed between ' + susp_solv['full_name'] + ' and 1-bromododecane.'

    side_prod_IL['ms'].append(prod_ms)
    side_prod_IL['name'].append(prod_name)

    side_prod_IL['msg'].append(prod_msg)

    return side_prod_IL

In [8]:
side_prod_SN2 = {

    'side_rx_type' : 'SN2',
    'ms' : [],
    'msg' : [],
    'name' : []

}

def side_rx_SN2(susp_solv):
    prod_name = susp_solv['name'] + '_' + 'BrAlkane' + '_SN2'
    prod_ms = susp_solv['ms'] + ms_BrAlk - 80.912
    prod_msg = 'SN2 product is formed between ' + susp_solv['full_name'] + ' and 1-bromododecane.'

    side_prod_SN2['ms'].append(prod_ms)
    side_prod_SN2['name'].append(prod_name)

    side_prod_SN2['msg'].append(prod_msg)

    return side_prod_SN2

In [9]:
side_prod_E2 = {

    'side_rx_type' : 'E2',
    'ms' : [],
    'msg' : [],
    'name' : []

}

def side_rx_E2(susp_solv):
    prod_name = susp_solv['name'] + '_' + 'BrAlkane' + '_E2'
    prod_ms = ms_BrAlk - 80.912 # One BrAlkane removes a HBr.
    prod_msg = 'E2 product is formed from by 1-bromododecane when reacting with ' + susp_solv['full_name'] + '.'

    side_prod_E2['ms'].append(prod_ms)
    side_prod_E2['name'].append(prod_name)

    side_prod_E2['msg'].append(prod_msg)

    return side_prod_E2

In [10]:
side_prod_EstAtk = {

    'side_rx_type' : 'EstAtk',
    'ms' : [],
    'msg' : [],
    'name' : []

}

#ester will be extracted from all_ms_Ol2Et

def side_rx_EstAtk(susp_solv, ester):

    for i, est_ms in enumerate(ester['ms']):
        
        prod_ms = est_ms + susp_solv['ms'] - 32.042 # remove a methanol from benzoate
        prod_name = susp_solv['name'] + '_' + ester['name'][i] + '_new_ester'
        prod_msg = 'The methyl ester fraction on the ' + ester['name'][i] + ' is replaced by ' + susp_solv['full_name'] + ' .'

        side_prod_EstAtk['ms'].append(prod_ms)
        side_prod_EstAtk['msg'].append(prod_msg)
        side_prod_EstAtk['name'].append(prod_name)
    
    return side_prod_EstAtk


In [11]:
side_prod_EstExg = {

    'side_rx_type' : 'EstExg',
    'ms' : [],
    'msg' : [],
    'name' : []

}

#ester will be extracted from all_ms_Ol2Et

def side_rx_EstExg(susp_solv, ester):

    # if susp_solv['name'] == 'EA':
    #     susp_solv['exg_frag_ms'] = 46.07 # this is the equivalent mass of alcohol to be replaced
    
    try:

        for i, est_ms in enumerate(ester['ms']):
            
            prod_ms = est_ms + susp_solv['exg_frag_ms'] - 32.042 # remove a methanol from benzoate
            prod_name = susp_solv['name'] + '_' + ester['name'][i] + '_new_ester'
            prod_msg = 'The methyl ester fraction on the ' + ester['name'][i] + ' is replaced by the equivalent alcohol part of the ' + susp_solv['full_name'] + ' .'

            side_prod_EstExg['ms'].append(prod_ms)
            side_prod_EstExg['msg'].append(prod_msg)
            side_prod_EstExg['name'].append(prod_name)
    
    except susp_solv['exg_frag_ms'] == None:

        print('This is not an ester.')


    return side_prod_EstExg

Based on the suspicious properties of solvents (submitted from the susp_solv_json), activate different side rxn mechanism and calculations.

In [12]:
for a, susp_solv in enumerate(susp_solv_data):

    if susp_solv['susp_prop'] == 'N_lp':

        side_prod_IL = side_rx_IL(susp_solv)
        side_prod_EstAtk = side_rx_EstAtk(susp_solv, all_ms_Ol2Et)
    
    elif susp_solv['susp_prop'] == 'OH':

        side_prod_SN2 = side_rx_SN2(susp_solv)
        side_prod_E2 = side_rx_E2(susp_solv)
        side_prod_EstAtk = side_rx_EstAtk(susp_solv, all_ms_Ol2Et)
    
    elif susp_solv['susp_prop'] == 'est':

        side_prod_EstExg = side_rx_EstExg(susp_solv, all_ms_Ol2Et)

#print(side_prod_IL, side_prod_EstAtk, side_prod_SN2, side_prod_E2, side_prod_EstExg)

Output

In [13]:
from datetime import datetime
now = datetime.now()

dt_string = now.strftime("%m%d%Y%H%M%S")
print(dt_string)

05072023120819


In [14]:
def decent_output_solv_side(side_rxn_log_dict):
    side_rxn_type = str(list(side_rxn_log_dict.keys())[0]) + ' : ' + side_rxn_log_dict['side_rx_type'] + '\n'
    f.write('========================== \n')
    f.write(side_rxn_type)

    len_val = len(list(side_rxn_log_dict['ms']))

    for i in range(len_val):
        grp_id = 'Group ' + str(i + 1) + ': \n'
        f.write('\n ------ \n')
        f.write(grp_id)
        f.write('\n')

        for k, key in enumerate(list(side_rxn_log_dict.keys())):

            if k != 0:        
            
                cont = str(key) + ' : ' + str(side_rxn_log_dict[key][i]) + '\n'
                f.write(cont)
    
    f.write('\n =========================== \n')


# def decent_output_std_subst_prod(std_subst_dict):
#     f.write('Standard substitution products to be expected: \n\n')

#     for k, key in enumerate(list(std_subst_dict.keys())):

#         for i, item in enumerate(list(std_subst_dict[key])):
            
#             grp_id = 'Group ' + str([k, i]) + ': \n'
#             cont = str(key) + ' : ' + str(item) + '\n\n'
#             f.write(grp_id)
#             f.write(cont)
    
#     f.write('=========================== \n')

def decent_output_std_hydrl_prod(std_hydrl_dict):
    # f.write('Hydrolysis products to be expected: \n\n')
    len_val = len(list(std_hydrl_dict['ms']))

    for i in range(len_val):
        grp_id = 'Group ' + str(i + 1) + ': \n'
        f.write('\n ------ \n')
        f.write(grp_id)

        for k, key in enumerate(list(std_hydrl_dict.keys())):        
            
            cont = str(key) + ' : ' + str(std_hydrl_dict[key][i]) + '\n'
            f.write(cont)
    
    f.write('\n =========================== \n')

    # for k, key in enumerate(list(std_hydrl_dict.keys())):

    #     grp_id = 'Group ' + str(k) + ': \n'
    #     cont = str(key) + ' : ' + str(std_hydrl_dict[key]) + '\n\n'
    #     f.write(grp_id)
    #     f.write(cont)
    
    # f.write('=========================== \n')


In [15]:
log_name = 'rxn_anls_log_' + str(dt_string) + '.txt'

with open(log_name, 'w') as f:
    
    f.write(str(now) + '\n')
    # decent_output_std_subst_prod(all_ms_Ol2Et)
    f.write('==================== \n')
    f.write('Potential side products not induced by solvents: \n')
    f.write('==================== \n')
    
    decent_output_std_hydrl_prod(all_ms_Est2Acd)

    # f.write('==================== \n')
    f.write('Potential side products induced by solvents: \n\n')

    all_solv_side = [side_prod_IL, side_prod_EstAtk, side_prod_SN2, side_prod_E2, side_prod_EstExg]

    for s, rxn in enumerate(all_solv_side):
        
        rxn['ms_with_Na'] = [x + 23.990 for x in rxn['ms']]

        decent_output_solv_side(rxn)
    
print('Please check: ' + str(log_name) + ' for output.')

Please check: rxn_anls_log_05072023120819.txt for output.


Below are irrelevant tests...

In [ ]:
atom_mass_dict = {
    'C' : 12.011,
    'H' : 1.008,
    'O' : 15.999,
    'N' : 14.007,
    'S' : 32.06,
    'Br' : 79.904
}



In [ ]:
rx_comb = {
    'CBr' : [['ACOH', [['H', -1], ['Br', -1]]], ['COH', [['H', -1], ['Br', -1]]], ['ACN', []]]
    
}

ind_solvent_from_db_DMF= {
    'name' : 'DMF',
    'formula' : [['C', 3], ['H', 7], ['N', 1], ['O', 1]],
    'basicity' : 0,
    'rx_site' : None
}

ind_solvent_from_db_NMI= {
    'name' : 'DMF',
    'formula' : [['C', 3], ['H', 7], ['N', 1], ['O', 1]],
    'kamlet_taft' : [0, 0.69, 0.88],
    'basicity' : 0,
    'rx_site' : [['ACN', 1]]
}

ind_sm_acoh_est = {
    'name' : 'ACOH_est',
    'formula' : [['C', 8], ['H', 8], ['O', 5]],
    'rx_site' : [['ACOH', 3], ['COOR', 1]]
}

ind_sm_br_alk = {
    'name' : 'Br_alk',
    'formula' : [['C', 12], ['H', 25], ['Br', 1]],
    'rx_site' : [['CBr', 1]]
}

In [ ]:
all_sm = ['ACOH_est', 'Br_alk']
all_solv = ['DMF']